In [ ]:
# 合併 predict 結果

import pandas as pd
import numpy as np

target = pd.read_csv('/home/ting/code/ehr_deidentification/formal_test_addi2b2/formal_all_split_lower_continue/predict_67210_epoch10_predict_regex_entity_imp.txt', keep_default_na=False, na_values=['NaN', 'null'],names=["file_id","label","start","end","entity","regular"], delimiter='\t',)
pivot = pd.read_csv('formal_test_addi2b2/formal_chunk_size_32/predict_66430_epoch10_predict_regex_entity_imp.txt',keep_default_na=False, na_values=['NaN', 'null'], names=["file_id","label","start","end","entity","regular"], delimiter='\t',)

pivot_entity_group = pivot.groupby('file_id')
target_entity_group = target.groupby('file_id')


finish_data = pd.DataFrame(columns=["file_id","label","start","end","entity","regular"])

for file_name in pivot_entity_group.groups.keys():
    # file_name='1032'
    specific_pivot_group = pivot_entity_group.get_group(file_name)
    specific_target_group = target_entity_group.get_group(file_name)
    
    same_start = specific_target_group.merge(specific_pivot_group, on=["start"], how='left')
    for idx, item in same_start.iterrows():
        # print(item.head())
        # print(item[['file_id_x','label_x','start','end_x','entity_x','regular_x']])
        # print(type(item))
        # print(item['end_y'],float('NaN'))
        if item['end_x'] != item['end_y'] and not np.isnan(item['end_y']):
            if len(str(item['entity_x']))>len(str(item['entity_y'])):
                new_item = item[['file_id_x','label_x','start','end_x','entity_x','regular_x']]
                new_item.index = ["file_id","label","start","end","entity","regular"]
                new_item['end'] = int(new_item['end'])
                # print(type(new_item['end']),new_item['end'])
                finish_data = finish_data.append(new_item)
            else:
                new_item = item[['file_id_y','label_y','start','end_y','entity_y','regular_y']]
                new_item.index = ["file_id","label","start","end","entity","regular"]
                new_item['end'] = int(new_item['end'])
                # print(type(new_item['end']),new_item['end'])
                finish_data = finish_data.append(new_item)
        else:
            new_item = item[['file_id_x','label_x','start','end_x','entity_x','regular_x']]
            new_item.index = ["file_id","label","start","end","entity","regular"]
            new_item['end'] = int(new_item['end'])
            # print(type(new_item['end']),new_item['end'])
            finish_data = finish_data.append(new_item)
        # break
    # break

finish_data.reset_index(drop=True, inplace=True)


# finish_data
finish_data.to_csv('formal_test_addi2b2/predict_67210_test_regex_imp_(combine_i2b2(no_all_split))_new.txt', sep='\t', index=False,header=None)



In [ ]:
# 合併 chatgpt 正規化日期以及 not add_i2b2 資料集的部分預測 label
import pandas as pd
import re
answer = pd.read_csv('formal_test_addi2b2/predict_67210_test_regex_imp_(combine_i2b2(no_all_split))_new.txt', names=["file_id", "label", "start", "end", "entity","regular"],keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t',)
n_t_p = pd.read_csv('/home/ting/code/ehr_deidentification/Need_chatgpt_date_fix.txt', names=["file_id", "label", "start", "end", "entity","regular"],keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t',)

# answer = answer[answer['label'] == 'DATE']
need_gpt_date = pd.DataFrame()
target_file = answer[answer['entity'].apply(lambda x: bool(re.findall(r"^[Nn][Oo][Ww]$", x)))]
need_gpt_date = pd.concat([need_gpt_date,target_file])
target_file = answer[answer['entity'].apply(lambda x: bool(re.findall(r"^[Tt][Oo][Dd][Aa][Yy]$", x)))]
need_gpt_date = pd.concat([need_gpt_date,target_file])
target_file = answer[answer['entity'].apply(lambda x: bool(re.findall(r"^[Pp][Rr][Ee][Vv][Ii][Oo][Uu][Ss]$", x)))]
need_gpt_date = pd.concat([need_gpt_date,target_file])

answer.drop(need_gpt_date.index.to_list(),inplace=True)
answer = pd.concat([answer,n_t_p])
answer = answer.sort_values(by=['file_id',"start"])
answer = answer.reset_index(drop=True)

# not add_i2b2 資料集的部分預測 label
pivot = pd.read_csv('formal_test/formal_chunk_size_32/predict_41348_test_regex_entity_imp.txt', keep_default_na=False, na_values=['NaN', 'null'],names=["file_id","label","start","end","entity","regular"], delimiter='\t',)

pivot_part1 = pivot[pivot['label']=="DURATION"]
pivot_part2 = pivot[pivot['label']=="PHONE"]
pivot_part3 = pivot[pivot['label']=="TIME"]
pivot = pd.concat([pivot_part1,pivot_part2,pivot_part3])
answer = answer[answer['label']!="DURATION"]
answer = answer[answer['label']!="PHONE"]
answer = answer[answer['label']!="TIME"]
answer = answer.reset_index(drop=True)
entity_group = answer.groupby('file_id')
# finish_data = pd.DataFrame(columns=["file_id","label","start","end","entity","regular"])
for idx,item in pivot.iterrows():
    # entity = location_other.group()
    start = int(item['start'])
    end = int(item['end'])

    answer.drop(entity_group.get_group(item['file_id']).query('start>=@start and end<=@end').index.tolist(),inplace=True)
    imp_entity = pd.DataFrame({
        "file_id":item['file_id'],
        "label":item['label'],
        "start":start,
        "end":end,
        "entity":item['entity'],
        "regular":item['regular']
    }, index=[0])
    # print(imp_entity)
    answer = answer.append(imp_entity, ignore_index=True)
answer = answer.sort_values(by=['file_id',"start"])
answer = answer.reset_index(drop=True)
# answer.to_csv("formal_test_addi2b2/formal_all_split_lower_continue/predict_67210_epoch10_predict_regex_(combine_i2b2(no_all_split))_ACE.txt", sep='\t', index=False,header=None)


In [ ]:
#刪除merge後重疊資料 可多跑幾次

import pandas as pd
import numpy as np

# answer = pd.read_csv('formal_test_addi2b2/predict_67210_test_regex_imp_(combine_i2b2(no_all_split))_new_ACE.txt', keep_default_na=False, na_values=['NaN', 'null'],names=["file_id","label","start","end","entity","regular"], delimiter='\t',)
answer_entity_group = answer.groupby('file_id')

finish_data = pd.DataFrame(columns=["file_id","label","start","end","entity","regular"])
for file_name in answer_entity_group.groups.keys():
    group = answer_entity_group.get_group(file_name)
    for i,(idx, item) in enumerate(group.iterrows()): 
        if i==0:
            finish_data = finish_data.append(item, ignore_index=True)
            pre_start=item['start']
            pre_end=item['end']
            pre_entity = item['label']
            continue
        elif item['start']>=pre_start and item['start']<pre_end:
            # print(item)                
            pre_start=item['start']
            pre_end=item['end']
            pre_entity = item['label']
        else:
            finish_data = finish_data.append(item, ignore_index=True)
            # print(item)
            pre_start=item['start']
            pre_end=item['end']
            pre_entity = item['label']
    # break
finish_data.to_csv("formal_test_addi2b2/predict_67210_test_regex_imp_(combine_i2b2(no_all_split))_new_ACE.txt", sep='\t', index=False,header=None)
